# Google Driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load test dataset

In [ ]:
import pandas as pd
import csv

In [ ]:
import os

def iterate_files_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            yield file_path

In [ ]:
# file_name = []
# start_pos = []
# origin_contents = []


# # 用法示例
# folder_path = "/content/drive/MyDrive/dataset/First_Phase_ReleaseCorrection/First_Phase_Release(Correction)/Validation_Release"
# for file_path in iterate_files_in_folder(folder_path):
#   # print(file_path.split('/')[-1])

#   with open(file_path, 'r', encoding='utf-8') as file:
#     contents = file.readlines()

#   curr_pos = 0

#   for content in contents:
#     if content != '\n':
#       flag = 0

#       temp_content = content.split('\n')[0]

#       while len(temp_content) >= 400:
#         file_name.append(file_path.split('/')[-1].split('.')[0])
#         start_pos.append(curr_pos)
#         origin_contents.append(temp_content[:400])
#         temp_content = temp_content[400:]
#         curr_pos += 400
#         flag = 1

#       if flag == 0:
#         file_name.append(file_path.split('/')[-1].split('.')[0])
#         start_pos.append(curr_pos)
#         origin_contents.append(content.split('\n')[0])
#         curr_pos += len(content)
#       else:
#         file_name.append(file_path.split('/')[-1].split('.')[0])
#         start_pos.append(curr_pos)
#         origin_contents.append(temp_content)
#         curr_pos += len(temp_content)

#     else:
#       curr_pos += 1

In [ ]:
file_name = []
start_pos = []
origin_contents = []

# folder_path = "/content/drive/MyDrive/dataset/Validation_Dataset_Answer/valid.tsv" # 驗證集路徑
folder_path = "/content/drive/MyDrive/dataset/opendid_test" # 測試集路徑
for file_path in iterate_files_in_folder(folder_path):

  with open(file_path, 'r', encoding='utf-8') as file:
    contents = file.readlines()

  curr_pos = 0

  for content in contents:
    if content != '\n':

      file_name.append(file_path.split('/')[-1].split('.')[0])
      start_pos.append(curr_pos)
      origin_contents.append(content.split('\n')[0])
      curr_pos += len(content)
    else:
      curr_pos += 1

In [ ]:
test_df = pd.DataFrame({
    'file_name': file_name,
    'start': start_pos,
    'content': origin_contents,
})
test_df

,file_name,start,content
0,1393,1,Episode No: 28D604157N
1,1393,25,281604.DNU
2,1393,37,"MCFATE, Jere IVAN"
3,1393,56,Lab No: 28D60415
4,1393,74,Harvest Park
...,...,...,...
47384,HSA0289,5440,"B. ""Bronchial trachea lymph node"". - Two (2..."
47385,HSA0289,5510,C. Hilar lymph node.
47386,HSA0289,5533,- One (1) negative lymph node.
47387,HSA0289,5566,D. Paratracheal lymph node.


In [ ]:
tsv_file_path = 'test.tsv'

# 使用 to_csv 函數保存 DataFrame 為 TSV 檔案
test_df.to_csv(tsv_file_path, sep='\t', index=False ,header=False)

# Model載入

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

plm = "EleutherAI/pythia-410m-deduped"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
PAD_IDX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
IGNORED_PAD_IDX = -100
PAD_IDX

50278

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(plm,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(plm, revision="step3000", config=config)
model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/911M [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [ ]:
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
  

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW


# loaded_optimizer = AdamW(model.parameters(),lr=5e-5)


# save_path = "/content/drive/MyDrive/410_10_model/my_model_7.pth"
# # save_path = '/content/drive/MyDrive/160_20_2_model/my_model_6.pth'
# # 載入模型和優化器的狀態
# checkpoint = torch.load(save_path)


# model.load_state_dict(checkpoint['model_state_dict'])
# loaded_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# prev_epoch = checkpoint['epoch']

In [ ]:
save_path = "/content/drive/MyDrive/410_13_model/my_model_7.pth" # model path

# load parameter into model
model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

# Predict

In [ ]:
from datasets import load_dataset, Features, Value
valid_data = load_dataset("csv", data_files="test.tsv", delimiter='\t',
                          features = Features({
                              'fid': Value('string'), 'idx': Value('int64'),
                              'content': Value('string'), 'label': Value('string')}),
                              column_names=['fid', 'idx', 'content', 'label'])
valid_list = list(valid_data['train'])
valid_list

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

[{'fid': '1393',
  'idx': 1,
  'content': 'Episode No:  28D604157N',
  'label': None},
 {'fid': '1393', 'idx': 25, 'content': '281604.DNU', 'label': None},
 {'fid': '1393', 'idx': 37, 'content': 'MCFATE, Jere IVAN ', 'label': None},
 {'fid': '1393', 'idx': 56, 'content': 'Lab No:  28D60415', 'label': None},
 {'fid': '1393', 'idx': 74, 'content': 'Harvest Park', 'label': None},
 {'fid': '1393', 'idx': 87, 'content': 'WANGARATTA    4415', 'label': None},
 {'fid': '1393', 'idx': 106, 'content': 'Specimen: Tissue', 'label': None},
 {'fid': '1393', 'idx': 123, 'content': 'D.O.B:  8/1/2011', 'label': None},
 {'fid': '1393', 'idx': 140, 'content': 'Sex:  F', 'label': None},
 {'fid': '1393',
  'idx': 148,
  'content': 'Collected: 13/05/2063 at 15:54',
  'label': None},
 {'fid': '1393',
  'idx': 179,
  'content': 'Location:  Tissue Pathology and Diagnostic Oncology-PORT BROUGHTON DISTRICT HOSPITAL AND HEALTH SERVICES',
  'label': None},
 {'fid': '1393',
  'idx': 284,
  'content': 'DR INES Ramon

In [ ]:
# find URL
def is_URL(input ,outputs):
  for t in input:
    try:
      if 'http://www' in t['content']:
        id = t['fid']

        sidx=int(t['idx'])
        lidx = t['content'].find('http://www')

        url_end = t['content'].find('\"')
        if url_end != -1:
          eidx = lidx + url_end
        else:
          eidx = sidx + len(t['content'])

        print()
        print(f"{id}\tURL\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx].split(')')[0]}")
        outputs.append(f"{id}\tURL\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}")

      elif 'http:/www' in t['content']:
        id = t['fid']

        sidx=int(t['idx'])
        lidx = t['content'].find('http:/www')

        url_end = t['content'].find('\"')
        if url_end != -1:
          eidx = lidx + url_end
        else:
          eidx = sidx + len(t['content'])

        print()
        print(f"{id}\tURL\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx].split(')')[0]}")
        outputs.append(f"{id}\tURL\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}")
    except TypeError:
        pass
  return outputs

In [ ]:
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 74.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=4c0f02f66fe1012ad46c4cd6dc249ed6602aa3f93e3115bd699d8885858dee3d
  Stored in directory: /root/.cache/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry


In [ ]:
import pycountry

all_countries = list(pycountry.countries)

country_name_list = []

# country_name_list.append('USA')
country_name_list.append('Australian')
country_name_list.append('Vietnam')

for i in all_countries:
  country_name_list.append(i.name)

country_name_list.remove('Congo')
country_name_list.remove('Mali')
country_name_list.remove('Benin')
country_name_list.remove('Oman')
country_name_list.remove('Chad')
country_name_list.remove('Israel')
country_name_list.remove('Jersey')
country_name_list.remove('Chile')

States_of_Australia = ['South Australia', 'Western Australia', 'Australian Capital Territory']

# find country
def is_COUNTRY(input ,outputs):
  for t in input:
    for cn in country_name_list:
      try:
        if cn in t['content']:
          if (States_of_Australia[0] not in t['content']) and (States_of_Australia[1] not in t['content']) and (States_of_Australia[2] not in t['content']):
            if cn == 'Australian':
              id = t['fid']
              sidx=int(t['idx'])
              lidx = t['content'].find(cn)
              eidx = lidx + len('Australia')

              print()
              print(f"{id}\tDURATION\t{lidx+sidx}\t{eidx+sidx}\t'Australia'")
              outputs.append(f"{id}\tCOUNTRY\t{lidx+sidx}\t{eidx+sidx}\t'Australia'")

            else:
              id = t['fid']
              sidx=int(t['idx'])
              lidx = t['content'].find(cn)
              eidx = lidx + len(cn)

              print()
              print(f"{id}\tCOUNTRY\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}")
              outputs.append(f"{id}\tCOUNTRY\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}")
      except TypeError:
        pass

  return outputs

In [ ]:
english_to_number = {
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "ten": 10,
    "eleven": 11,
    "twelve": 12,
    "thirteen": 13,
    "fourteen": 14,
    "fifteen": 15,
    "sixteen": 16,
    "seventeen": 17,
    "eighteen": 18,
    "nineteen": 19,
    "twenty": 20,
    "twenty-one": 21,
    "twenty-two": 22,
    "twenty-three": 23,
    "twenty-four": 24,
    "twenty-five": 25,
    "twenty-six": 26,
    "twenty-seven": 27,
    "twenty-eight": 28,
    "twenty-nine": 29,
    "thirty": 30,
    "thirty-one": 31,
    "thirty-two": 32,
    "thirty-three": 33,
    "thirty-four": 34,
    "thirty-five": 35,
    "thirty-six": 36,
    "thirty-seven": 37,
    "thirty-eight": 38,
    "thirty-nine": 39,
    "forty": 40,
    "forty-one": 41,
    "forty-two": 42,
    "forty-three": 43,
    "forty-four": 44,
    "forty-five": 45,
    "forty-six": 46,
    "forty-seven": 47,
    "forty-eight": 48,
    "forty-nine": 49,
    "fifty": 50,
    "fifty-one": 51,
    "fifty-two": 52,
    "fifty-three": 53,
    "fifty-four": 54,
    "fifty-five": 55,
    "fifty-six": 56,
    "fifty-seven": 57,
    "fifty-eight": 58,
    "fifty-nine": 59,
    "sixty": 60,
    "sixty-one": 61,
    "sixty-two": 62,
    "sixty-three": 63,
    "sixty-four": 64,
    "sixty-five": 65,
    "sixty-six": 66,
    "sixty-seven": 67,
    "sixty-eight": 68,
    "sixty-nine": 69,
    "seventy": 70,
    "seventy-one": 71,
    "seventy-two": 72,
    "seventy-three": 73,
    "seventy-four": 74,
    "seventy-five": 75,
    "seventy-six": 76,
    "seventy-seven": 77,
    "seventy-eight": 78,
    "seventy-nine": 79,
    "eighty": 80,
    "eighty-one": 81,
    "eighty-two": 82,
    "eighty-three": 83,
    "eighty-four": 84,
    "eighty-five": 85,
    "eighty-six": 86,
    "eighty-seven": 87,
    "eighty-eight": 88,
    "eighty-nine": 89,
    "ninety": 90,
    "ninety-one": 91,
    "ninety-two": 92,
    "ninety-three": 93,
    "ninety-four": 94,
    "ninety-five": 95,
    "ninety-six": 96,
    "ninety-seven": 97,
    "ninety-eight": 98,
    "ninety-nine": 99,
    "one hundred": 100,
}

In [ ]:
# 檢查缺漏，最後再跟answer比較
import re

duration_key = "ago"

year_key = ['years', 'yrs']
month_key = ['months']
week_key = ['wks', 'weeks', 'wk']

num_dict = {}

# find duration
def is_DURATION(input, temp_outputs):
  for t in input:
    try:
      words = re.findall(r'\b\w+\b', t['content'])

      while duration_key in words:
        ago_id = words.index(duration_key)
        duration_start_word = words[ago_id - 2]
        time_quantifier = words[ago_id - 1]

        if duration_start_word != 'many' and duration_start_word != 'few':
          lidx = t['content'].find(duration_start_word)
          eidx = lidx + len(duration_start_word) + len(time_quantifier) + 1 # 三個字加兩個空格

          id = t['fid']
          sidx=int(t['idx'])

          if time_quantifier in year_key:
            YMW = 'Y'
          elif time_quantifier in month_key:
            YMW = 'M'
          else:
            YMW = 'W'

          try:
            NUM = int(duration_start_word)
          except ValueError:
            if duration_start_word in english_to_number:
              NUM = english_to_number[duration_start_word]
            else:
              _ = words.pop(ago_id)
              continue


          print()
          print(f"{id}\tDURATION\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}\tP{NUM}{YMW}")
          temp_outputs.append(f"{id}\tDURATION\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}\tP{NUM}{YMW}\n")

        _ = words.pop(ago_id)

    except:
      pass

  return temp_outputs



# ip = [{'fid': '1429', 'idx': 1, 'content': 'Additional history from Dr BAILEY Bacayo, testicular tumour forty wks ago. Recent biopsy at NARACOORTE - sarcoma.', 'label': None},
#  {'fid': '1429', 'idx': 14, 'content': 'Left parietal tumour initially ? meningioma.  Now ? GBM.  Also ? met (gastric ca 8 years ago).', 'label': None},
#  {'fid': '1429', 'idx': 31, 'content': 'Intra op - ?peritoneal cancer.  Breast ca, GORD, OA, Hep B many years ago.  Laparotomy - inflamed appendix; small cystic mass right adnexa.', 'label': None}]
# op = []
# op = is_DURATION(ip, op)
# op

In [ ]:
set_key = "twice"

# find set
def is_SET(input, temp_outputs):
  for t in input:
    try:
      if set_key in t['content']:
        id = t['fid']
        sidx=int(t['idx'])

        lidx = t['content'].find(set_key)
        eidx = lidx + len(set_key)


        print()
        print(f"{id}\tSET\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}\tR2")
        temp_outputs.append(f"{id}\tSET\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}\tR2\n")
    except:
      pass

  return temp_outputs

# ip = [{'fid': '1429', 'idx': 1, 'content': 'Additional history from Dr BAILEY Bacayo, testicular tumour forty wks ago. Recent biopsy at NARACOORTE - sarcoma.', 'label': None},
#  {'fid': '1429', 'idx': 14, 'content': 'Number found:   11 (specimen examined twice for lymph nodes)', 'label': None},
#  {'fid': '1429', 'idx': 31, 'content': 'Intra op - ?peritoneal cancer.  Breast ca, GORD, OA, Hep B many years ago.  Laparotomy - inflamed appendix; small cystic mass right adnexa.', 'label': None}]
# op = []
# op = is_SET(ip, op)
# op

In [ ]:
loc_key = ["PO BOX", "PO Box"]

# find location-other
def is_Loc(input, temp_outputs):
  for t in input:
    try:
      if loc_key[0] in t['content']:

        words = re.findall(r'\b\w+\b', t['content'])

        id = words.index("PO")
        num = words[id + 2]

        id = t['fid']
        sidx=int(t['idx'])

        lidx = t['content'].find(loc_key[0])
        eidx = lidx + len(loc_key[0]) + len(num) + 1

        print()
        print(f"{id}\tLOCATION-OTHER\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}")
        temp_outputs.append(f"{id}\tLOCATION-OTHER\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}\n")

      elif loc_key[1] in t['content']:
        words = re.findall(r'\b\w+\b', t['content'])

        id = words.index("PO")
        num = words[id + 2]

        id = t['fid']
        sidx=int(t['idx'])

        lidx = t['content'].find("PO")
        eidx = lidx + len(loc_key[0]) + len(num) + 1

        print()
        print(f"{id}\tLOCATION-OTHER\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}")
        temp_outputs.append(f"{id}\tLOCATION-OTHER\t{lidx+sidx}\t{eidx+sidx}\t{t['content'][lidx:eidx]}\n")
    except:
      pass

  return temp_outputs

In [ ]:
from tqdm import tqdm#, tqdm_notebook



tokenizer.padding_side = 'left'
def sample_batch(model, tokenizer, input):
    """Generate text from a trained model."""
    model.eval()
    seeds = [f"{bos} {text['content']} {sep}" for text in input]

    texts = tokenizer(seeds, return_tensors = 'pt', padding=True).to(device)
    outputs = []

    outputs = is_URL(input, outputs)
    outputs = is_COUNTRY(input, outputs)
    duration_temp = is_DURATION(input)
    set_temp = is_SET(input)


    #return
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**texts, max_new_tokens=400, pad_token_id = PAD_IDX,
                                        eos_token_id=tokenizer.convert_tokens_to_ids(eos))
        preds = tokenizer.batch_decode(output_tokens)
        for idx , pred in enumerate(preds):
            pred = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
            if pred == "PHI: NULL":
                continue
            phis = pred.split('\n')
            lidxs = {}
            for p in phis:
                tid = p.find(':')
                if tid > 0:
                    text = p[tid+1:].strip()
                    nv = text.find('=>')
                    normalizedV = None

                    if nv != -1:
                      normalizedV = text.split('>')[-1]
                      text = text.split('=')[0]


                    lidx = 0
                    if text in lidxs:
                        lidx = lidxs[text]
                    lidx = input[idx]['content'].find(text, lidx)
                    eidx = lidx+len(text)
                    lidxs[text] = eidx
                    sidx=int(input[idx]['idx'])
                    if normalizedV is None:
                        outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}')
                    else:
                        outputs.append(f'{input[idx]["fid"]}\t{p[:tid]}\t{lidx+sidx}\t{eidx+sidx}\t{text}\t{normalizedV}')
    return outputs


duration_temp = []
set_temp = []
f = open("answer.txt", "w", encoding='utf-8')
BATCH_SIZE = 25
for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
    with torch.no_grad():
        seeds = valid_list[i:i+BATCH_SIZE]
        outputs = sample_batch(model, tokenizer, input=seeds)
        for o in outputs:
            f.write(o)
            f.write('\n')
f.close()

NameError: ignored

In [ ]:
from tqdm import tqdm#, tqdm_notebook

duration_temp = []
set_temp = []
loc_temp = []
BATCH_SIZE = 25

for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
  input = valid_list[i:i+BATCH_SIZE]
  duration_temp = is_DURATION(input, duration_temp)
  set_temp = is_SET(input, set_temp)
  loc_temp = is_Loc(input, loc_temp)

 47%|████▋     | 896/1896 [00:00<00:00, 4491.15it/s]


1198	DURATION	400	407	8 years	P8Y

2070	SET	2688	2693	twice	R2

2077	SET	2423	2428	twice	R2

2387	SET	3951	3956	twice	R2

2418	DURATION	145	151	15 yrs	P15Y

2418	DURATION	202	208	15 yrs	P15Y

2465	LOCATION-OTHER	68	79	PO BOX 1061

5143	DURATION	423	434	forty years	P40Y

5243	LOCATION-OTHER	502	513	PO Box 1013

5349	DURATION	459	467	6 months	P6M

5349	DURATION	482	489	3 weeks	P3W

5349	DURATION	509	513	1 wk	P1W

file32052	SET	4146	4151	twice	R2


100%|██████████| 1896/1896 [00:00<00:00, 4949.93it/s]


file56166	DURATION	296	304	19 years	P19Y


# Post-processing

In [ ]:
import re

def compare(curr, prev):
  if curr[2] == prev[2] and curr[3] == prev[3]:
    return 1
  else:
    if curr[4] == prev[4]:
      if curr[2] < prev[3]:
        return 1
      else:
        return 0
    else:
      return 0

def time_normalize_check(line):
  try:
    exist = line.split('\t')[5]
  except IndexError:
    return 1
  return 0

In [ ]:
# 指定舊檔案和新檔案的檔案路徑
old_file_path = 'answer.txt'
new_file_path = 'process_ans.txt'
error_file_path = "error.txt"



max_flag = 0 # 處理太長的
max_over_len = 100
idx_flag = 0 # 處理沒有content
compare_flag = 0
tn_check_flag = 0

label_list = ["IDNUM", "MEDICALRECORD", "PATIENT", "CITY", "STATE", "ZIP", "DATE", "DEPARTMENT", "HOSPITAL", "DOCTOR",
        "TIME", "STREET", "ORGANIZATION", "AGE", "DURATION", "PHONE", "COUNTRY", "LOCATION-OTHER", 'URL', 'SET']
label_flag = 0



# 逐行讀取舊檔案，並將每一行寫入新檔案
with open(old_file_path, 'r', encoding='utf-8') as old_file, open(new_file_path, 'w', encoding='utf-8') as new_file, open(error_file_path, 'w', encoding='utf-8') as error_file:
    for i, line in enumerate(old_file):

      try:
        curr_content = line.strip().split('\t')

        if(len(line.strip().split('\t')[4]) > max_over_len):
          max_flag = 1

        if line.strip().split('\t')[1] not in label_list:
          label_flag = 1

      except IndexError:
        idx_flag = 1


      if i != 0:
        if idx_flag == 0:
          compare_flag = compare(curr_content, prev_content)

        if line.strip().split('\t')[1] == 'DATE' or line.strip().split('\t')[1] == 'TIME' or line.strip().split('\t')[1] == 'DURATION' or line.strip().split('\t')[1] == 'SET':
          tn_check_flag = time_normalize_check(line)

        if (idx_flag == 0) and (compare_flag == 0) and (max_flag == 0) and (tn_check_flag == 0) and (label_flag == 0):
          new_file.write(line)
          prev_content = curr_content
        else:
          error_file.write(line)
      else:
        new_file.write(line)
        prev_content = curr_content

      idx_flag = 0
      max_flag = 0
      compare_flag = 0
      tn_check_flag = 0
      label_flag = 0

In [ ]:
old_file_path = 'process_ans.txt'

with open(old_file_path, 'r', encoding='utf-8') as old_file:
  contents = old_file.readlines()

duration_model_list = []
set_model_list = []

for content in contents:
  if content.strip().split('\t')[1] == "DURATION":
    duration_model_list.append(content)
  elif content.strip().split('\t')[1] == "SET":
    set_model_list.append(content)



new_duration = []
model_list_index = []
for dt in duration_temp:
  flag = 0
  for i, dm in enumerate(duration_model_list):
    if dt.split('\t')[0] == dm.split('\t')[0]:
      if dt.split('\t')[2] == dm.split('\t')[2]:
        new_duration.append(dt)
        model_list_index.append(i)
        flag = 1
  if flag == 0:
    new_duration.append(dt)

for j in range(len(duration_model_list)):
  if j not in model_list_index:
    new_duration.append(duration_model_list[j])



new_set = []
model_list_index = []
for st in set_temp:
  flag = 0
  for i, sm in enumerate(set_model_list):
    if st.split('\t')[0] == sm.split('\t')[0]:
      if st.split('\t')[2] == sm.split('\t')[2]:
        new_set.append(st)
        model_list_index.append(i)
        flag = 1
  if flag == 0:
    new_set.append(st)

for j in range(len(set_model_list)):
  if j not in model_list_index:
    new_set.append(set_model_list[j])

In [ ]:
new_duration

['1198\tDURATION\t400\t407\t8 years\tP8Y\n',
 '2418\tDURATION\t145\t151\t15 yrs\tP15Y\n',
 '2418\tDURATION\t202\t208\t15 yrs\tP15Y\n',
 '5143\tDURATION\t423\t434\tforty years\tP40Y\n',
 '5349\tDURATION\t459\t467\t6 months\tP6M\n',
 '5349\tDURATION\t482\t489\t3 weeks\tP3W\n',
 '5349\tDURATION\t509\t513\t1 wk\tP1W\n',
 'file56166\tDURATION\t296\t304\t19 years\tP19Y\n',
 '1633\tDURATION\t324\t332\t2 months\tP2M\n',
 '1633\tDURATION\t375\t384\t18 months\tP18M\n',
 '2311\tDURATION\t363\t365\t2w\tP2W\n',
 'file43664\tDURATION\t341\t347\t6/1/67\t2067-06-01\n']

In [ ]:
new_set

['2070\tSET\t2688\t2693\ttwice\tR2\n',
 '2077\tSET\t2423\t2428\ttwice\tR2\n',
 '2387\tSET\t3951\t3956\ttwice\tR2\n',
 'file32052\tSET\t4146\t4151\ttwice\tR2\n']

In [ ]:
new_duration_file_name = []
for d in new_duration:
  new_duration_file_name.append(d.split('\t')[0])

new_set_file_name = []
for s in new_set:
  new_set_file_name.append(s.split('\t')[0])

new_loc_file_name = []
for l in loc_temp:
  new_loc_file_name.append(l.split('\t')[0])

In [ ]:
new_duration_file_name

['1198',
 '2418',
 '2418',
 '5143',
 '5349',
 '5349',
 '5349',
 'file56166',
 '1633',
 '1633',
 '2311',
 'file43664']

In [ ]:
old_file_path = "process_ans.txt"
new_file_path = "process2_ans.txt"

new_duration_count = 0
d_flag = 0
new_set_count = 0
s_flag = 0
new_loc_count = 0
l_flag = 0

new_duration_len = len(new_duration)
new_set_len = len(new_set)
new_loc_len = len(loc_temp)

with open(old_file_path, 'r', encoding='utf-8') as old_file, open(new_file_path, 'w', encoding='utf-8') as new_file:
  for i, line in enumerate(old_file):
    if line.strip().split('\t')[1] != "DURATION" and line.strip().split('\t')[1] != "SET" and line.strip().split('\t')[1] != "LOCATION-OTHER":
      new_file.write(line)

    if line.strip().split('\t')[0] in new_duration_file_name and d_flag == 0:
      id = new_duration_file_name.index(line.strip().split('\t')[0])
      print(new_duration[id])
      new_file.write(new_duration[id])
      _ = new_duration_file_name.pop(id)
      _ = new_duration.pop(id)
      new_duration_count += 1
      if new_duration_count >= new_duration_len:
        d_flag = 1

    if line.strip().split('\t')[0] in new_set_file_name and s_flag == 0:
      id = new_set_file_name.index(line.strip().split('\t')[0])
      new_file.write(new_set[id])
      _ = new_set_file_name.pop(id)
      _ = new_set.pop(id)
      new_set_count += 1
      if new_set_count >= new_set_len:
        s_flag = 1

    if line.strip().split('\t')[0] in new_loc_file_name and l_flag == 0:
      id = new_loc_file_name.index(line.strip().split('\t')[0])
      new_file.write(loc_temp[id])
      _ = new_loc_file_name.pop(id)
      _ = loc_temp.pop(id)
      new_loc_count += 1
      if new_loc_count >= new_loc_len:
        l_flag = 1




    # elif line.strip().split('\t')[1] == "DURATION":
    #   try:
    #     id = new_duration_file_name.index(line.strip().split('\t')[0])
    #     new_file.write(new_duration[id])
    #   except ValueError:
    #     pass
    # elif line.strip().split('\t')[1] == "SET":
    #   try:
    #     id = new_set_file_name.index(line.strip().split('\t')[0])
    #     new_file.write(new_set[id])
    #   except ValueError:
    #     pass

# with open(new_file_path, 'r', encoding='utf-8') as new_file:
#   lines = new_file.readlines()

# temp_lines = lines
# for i, line in enumerate(lines):
#   if line.strip().split('\t')[0] in new_duration_file_name:
#     id = new_duration_file_name.index(line.strip().split('\t')[0])
#     new_line = new_duration[id]
#     temp_lines.insert(i, new_line + "\n")
#   elif line.strip().split('\t')[0] in new_set_file_name:
#     id = new_set_file_name.index(line.strip().split('\t')[0])
#     new_line = new_set[id]
#     temp_lines.insert(i, new_line + "\n")

# new_file_path = "process3_ans.txt"
# with open(new_file_path, 'w', encoding='utf-8') as new_file:
#   new_file.writelines(temp_lines)

# for d in new_duration:
#   new_file.write(d)
#   new_file.write('\n')

# for s in new_set:
#   new_file.write(s)
#   new_file.write('\n')

1198	DURATION	400	407	8 years	P8Y

1633	DURATION	324	332	2 months	P2M

1633	DURATION	375	384	18 months	P18M

2311	DURATION	363	365	2w	P2W

2418	DURATION	145	151	15 yrs	P15Y

2418	DURATION	202	208	15 yrs	P15Y

5143	DURATION	423	434	forty years	P40Y

5349	DURATION	459	467	6 months	P6M

5349	DURATION	482	489	3 weeks	P3W

5349	DURATION	509	513	1 wk	P1W

file43664	DURATION	341	347	6/1/67	2067-06-01

file56166	DURATION	296	304	19 years	P19Y



In [ ]:
import pandas as pd
old_file_path = 'process2_ans.txt'


my_df = pd.read_csv(old_file_path, sep='\t',header=None,names=['file_name', 'label', 'start', 'end', 'content', 'time_normalized'])
# ans_df = pd.read_csv('/content/valid_ans.txt', sep='\t',header=None,names=['file_name', 'label', 'start', 'end', 'content', 'time_normalized'])

def merge_content(row):
    if pd.notna(row['time_normalized']):
        row['content'] = row['content'] + '=>' + row['time_normalized']
    return row['content']


my_df['content'] = my_df.apply(merge_content, axis=1)
my_df = my_df.drop(columns=['time_normalized'])

In [ ]:
old_file_path = 'process2_ans.txt'
new_file_path = 'process3_ans.txt'
error_file_path = 'error3.txt'

my_file_list = my_df['file_name'].unique()

error_flag = 0
patient_count = 0
patient_pos = 0
curr_file = 0

with open(old_file_path, 'r', encoding='utf-8') as old_file, open(new_file_path, 'w', encoding='utf-8') as new_file, open(error_file_path, 'w', encoding='utf-8') as error_file:
  for i, line in enumerate(old_file):

    file_id = line.strip().split('\t')[0]
    if file_id == my_file_list[curr_file]:
      file_label = line.strip().split('\t')[1]

      if file_label == 'PATIENT':
        patient_count += 1
        if patient_count == 1:
          patient_pos = int(line.strip().split('\t')[3])
        else:
          not_patient_pos = int(line.strip().split('\t')[3])
          if (not_patient_pos - patient_pos) < 100:
            line = line.replace("PATIENT", "STREET")
            print(line)
            error_flag = 0
          else:
            error_flag = 0
      if error_flag == 0:
        new_file.write(line)
      else:
        error_file.write(line)

      error_flag = 0
    else:
      new_file.write(line)
      curr_file += 1
      patient_count = 0
      patient_pos = 0

1537	STREET	82	89	Cedrela

1198	STREET	73	77	Wren

2081	STREET	74	81	Serrano

1618	STREET	71	79	Vernalis

2096	STREET	73	79	Giusti

2257	STREET	72	77	Deyne

2269	STREET	72	86	Spectacle Pond

2287	STREET	73	79	Adolph

2279	STREET	82	89	VILLA 9

2282	STREET	69	80	Shakespeare

2285	STREET	71	76	Siren

2262	STREET	72	82	Hazeldean,

2276	STREET	68	74	Gazebo

2284	STREET	81	89	Bungowen

2259	STREET	75	83	Rembrant

2346	STREET	71	77	Bruell

2327	STREET	86	94	Felsberg

2311	STREET	69	76	Miggins

2360	STREET	81	91	Shackliffe

2363	STREET	91	102	Wm Clifford

2361	STREET	83	87	Nook

2387	STREET	78	85	Durbach

2379	STREET	95	101	Vanden

2365	STREET	78	84	Verdin

2359	STREET	68	78	St Norbert

2405	STREET	89	97	Scratton

2417	STREET	81	87	Eggers

2444	STREET	78	86	Anacapri

2446	STREET	78	86	Limetree

2424	STREET	70	80	Frenchmans

2404	STREET	79	86	Dornoch

2422	STREET	84	94	Appomattox

2473	STREET	67	73	Calyer

2457	STREET	81	90	Harrishof

2474	STREET	79	86	Marieba

2463	STREET	70	73	Fir

2512	STRE

In [ ]:
old_file_path = 'process3_ans.txt'
with open(old_file_path, 'r', encoding='utf-8') as old_file:
  contents = old_file.readlines()

new_contents = []

for i, content in enumerate(contents):
  change_flag = 0
  if content.strip().split('\t')[1] == 'LOCATION-OTHER':
    if content.strip().split('\t')[4][0] != 'P':
      print(content.replace("LOCATION-OTHER", "STREET"))
      new_contents.append(content.replace("LOCATION-OTHER", "STREET"))
      change_flag = 1

  if content.strip().split('\t')[1] == 'DURATION':
    if content.strip().split('\t')[5][0] != 'P':
      print(content.replace("DURATION", "DATE"))
      new_contents.append(content.replace("DURATION", "DATE"))
      change_flag = 1

  if content.strip().split('\t')[1] == 'URL':
    try:
      # content.strip().split('\t')[4].split(')')[1]




      ori_end = content.strip().split('\t')[3]
      temp = content.replace(content.strip().split('\t')[4], content.strip().split('\t')[4].split(')')[0])
      temp = temp.replace(ori_end, str(int(content.strip().split('\t')[2]) + len(content.strip().split('\t')[4].split(')')[0])))
      print(temp)
      new_contents.append(temp)
      change_flag = 1
    except:
      change_flag = 0


  if content.strip().split('\t')[1] == 'DATE':
    if content.strip().split('\t')[5][0] == 'P':
      print(f"DATE ERROR: {content}")
      change_flag = 1

  if content.strip().split('\t')[1] == 'TIME':
    if content.strip().split('\t')[4][0] == 'A':
      print(f"TIME ERROR: {content}")
      change_flag = 1

  if content.strip().split('\t')[1] == 'DEPARTMENT':
    if content.strip().split('\t')[4] == 'ACT':
      print(content.replace("DEPARTMENT", "STATE"))
      new_contents.append(content.replace("DEPARTMENT", "STATE"))
      change_flag = 1

  if change_flag == 0:
    new_contents.append(content)


new_file_path = "process4_ans.txt"

with open(new_file_path, 'w', encoding='utf-8') as new_file:
  for new_content in new_contents:
    new_file.write(new_content)

DATE ERROR: 1192	DATE	208	211	6cm	P6W

1977	STATE	209	212	ACT

1975	URL	4094	4181	http://www.sequenom.comlhome/productsservices/genetic/analysis/applications/oncocartal/

2098	STATE	179	182	ACT

2335	STATE	199	202	ACT

2417	STATE	5116	5119	ACT

2512	STATE	175	178	ACT

2495	STATE	3232	3235	ACT

5123	URL	8931	8973	http://www.mdanderson.org/breastcancer_RCB

5213	STATE	197	200	ACT

5230	STATE	216	219	ACT

5243	STATE	220	223	ACT

5349	STATE	192	195	ACT

file43664	DATE	341	347	6/1/67	2067-06-01

TIME ERROR: file51190	TIME	5420	5459	ASCO/CAP recommendations 2013,2105-2122	2064-02-10T22: 2

TIME ERROR: file56947	TIME	6146	6185	ASCO/CAP recommendations 2013,2105-2122	2064-02-10T22: 2

TIME ERROR: file56166	TIME	4735	4774	ASCO/CAP recommendations 2013,2105-2122	2064-02-10T22: 2



# "process4_ans" is final answer file